<a href="https://colab.research.google.com/github/aditi-dam/hackbca2022/blob/master/Hack_BCA_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [240]:
import numpy as np

import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

In [241]:
mbti_df = pd.read_csv("mbti_1.csv")

mind_names = ["E" , "I"]
mind = []  # E or I

energy_names = ["N",  "S"]
energy = [] # N or S

nature_names = ["F", "T"]
nature = [] # F or T

tactics_names = ["J", "P"]

tactics = [] # J OR P

for t in mbti_df.type:
  mind.append(mind_names.index(t[0]))
  energy.append(energy_names.index(t[1]))
  nature.append(nature_names.index(t[2]))
  tactics.append(tactics_names.index(t[3]))

mbti_df['mind'] = mind
mbti_df['energy'] = energy
mbti_df['nature'] = nature
mbti_df['tactics'] = tactics



In [242]:
mbti_df

,type,posts,mind,energy,nature,tactics
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,1,0,0,0
1,ENTP,'I'm finding the lack of me in these posts ver...,0,0,1,1
2,INTP,'Good one _____ https://www.youtube.com/wat...,1,0,1,1
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",1,0,1,0
4,ENTJ,'You're fired.|||That's another silly misconce...,0,0,1,0
...,...,...,...,...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...,1,1,0,1
8671,ENFP,'So...if this thread already exists someplace ...,0,0,0,1
8672,INTP,'So many questions when i do these things. I ...,1,0,1,1
8673,INFP,'I am very conflicted right now when it comes ...,1,0,0,1


In [243]:
from sklearn.model_selection import train_test_split

label_mind = mbti_df.mind
label_energy = mbti_df.energy
label_nature = mbti_df.nature
label_tactics = mbti_df.tactics
feature = mbti_df.posts

feature_train, feature_test, labelm_train, labelm_test = train_test_split (feature, label_mind, test_size =.3 , random_state= 42, stratify= label_mind)
feature_train, feature_test, labele_train, labele_test = train_test_split (feature, label_energy, test_size =.3 , random_state= 42, stratify= label_energy)
feature_train, feature_test, labeln_train, labeln_test = train_test_split (feature, label_nature, test_size =.3 , random_state= 42, stratify= label_nature)
feature_train, feature_test, labelt_train, labelt_test = train_test_split (feature, label_tactics, test_size =.3 , random_state= 42, stratify= label_nature)


In [244]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_4 (Dense)             (None, 16)                816       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [245]:
feature_train

5703    wisdom of the sages gods beyond bible pages sn...
7984    'I enjoy RPGs and got so used to being able to...
3234    'Personally I'd change that into   Some people...
7908    'INFJ male here in love with and mutual attrac...
277     'I can't say much really as none of this truly...
                              ...                        
5178    'Can anyone take a guess at the types of the 3...
6449    YES! For some reason my signature isn't workin...
4466    'INTJs tend to be much cooler and logical abou...
1210    Yeah, it makes sense and that's what u gave be...
1293    'I need hugs.   Been in love with an ENFJ for ...
Name: posts, Length: 6072, dtype: object

In [246]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])


In [247]:
feature_val = feature_train[3036:]
partial_feature_train = feature_train[:3036]


In [248]:
m_val = labelm_train[3036:]
partial_m_train = labelm_train[:3036]

e_val = labele_train[3036:]
partial_e_train = labele_train[:3036]

n_val = labeln_train[3036:]
partial_n_train = labeln_train[:3036]

t_val = labelt_train[3036:]
partial_t_train = labelt_train[:3036]

In [249]:
def normalize(res, range):
  normalized_vals = []
  for arr in res:
    normalized_vals.append((arr[0] + abs(minima))/range)
  return normalized_vals

In [250]:
def float_to_mind(float_results):
  res = []
  for num in float_results:
    if(num < 0.5):
      res.append("E")
    else:
      res.append("I")
  return res

In [251]:
def float_to_energy(float_results):
  res = []
  for num in float_results:
    if(num < 0.5):
      res.append("N")
    else:
      res.append("S")
  return res

In [252]:
def float_to_nature(float_results):
  res = []
  for num in float_results:
    if(num < 0.5):
      res.append("F")
    else:
      res.append("T")
  return res

In [253]:
def float_to_tactics(float_results):
  res = []
  for num in float_results:
    if(num < 0.5):
      res.append("J")
    else:
      res.append("P")
  return res

In [254]:
input_string = "'Dear INTP,   I enjoyed our conversation the other day.  Esoteric gabbing about the nature of the universe and the idea that every rule and social code being arbitrary constructs created...|||Dear ENTJ sub,   Long time no see.  Sincerely, Alpha|||None of them. All other types hurt in deep existential ways that I want no part of.|||Probably a sliding scale that depends on individual preferences, like everything in humanity.|||Draco Malfoy also. I'd say he's either 358 or 368.|||I'm either 358 or 385, though in which stacking to me is a somewhat arbitrary distinction to make as I believe that the core indicates primary motivation and has a hand in every action. Therefore, a...|||I'm not particularly introverted or extraverted, personally. That said, I would say I'm somewhat unphased by either social interactions or being alone. What I'd say I crave more so than anything is...|||Dear Type 9 INFP,  Your absolute admiration of me is refreshing. You're a great girlfriend and I wish we both didn't have such busy schedules so we could be around one another more often.  Keep...|||2% still means about 1/50 people. I've probably seen 1-2 others today. I never understood fascination by virtue of rarity.|||So, you're on the ESFJ train also, right?|||I have toyed with the idea of the OP being an extrovert also for awhile now, actually. After many conversations with him, however I'm disinclined to believe it due to OP being much too close with Fi...|||Still ESFJ|||I disagree.  Definite ESFJ. Fe- Si ALL up in this.|||Where have you been?  Your mother and I have been worried sick.|||Similar feelings concerning ENTPs.|||I collect shoes. I do so because I like status and nothing communicates such a thing as much as a pair of Jordans.|||Sure. Let's get weird.    Back. Off.|||Best...|||[Insert other into previous post.]|||Wow, don't nobody got time fo' dat...  Jk, u kno u mah boi.|||Well, as I'm sure everyone knows, being social primary doesn't necessarily equal being social. I like other people just fine, but lately I've been on a bit of a break when it comes to my...|||That's pretty crappy of her. Your sex life is definitely not her business. If I were you, I'd definitely talk to her about it.|||Yeah, I had a lot longer to think about it. That said, I don't think you're Te, so if we operate under the assumptions in the OP, you're INFJ because nothing else makes sense. I'm not saying I...|||Whether or not you actually are, your mom makes fun of you for being a virgin? Is this playful teasing?|||Just stab in the dark here, I don't think that's Te under the hood. If this here is all I'm given to work with, I'd have to say INFJ.  Compare your idea of 'logic' to mine:  Logic is a series of...|||What makes the most sense?|||Tentatively INTJ. Still reading.  EDIT: So far, I am of the belief that you're most of the things you say you are. I appreciate the level of detail.  EDIT: Also relatively certain you are Ni...|||Refreshing self with reading about instinctual variants.  God, I'm a hard social type.|||WOW! Useful thought you had there! Wanna know what my favorite part was?  The fact that it ended...|||I am both annoyed and intrigued by xxFPs.  As far as I can tell, I dislike all J types in my normal life.|||I'm not of the mind that any enneatype/MBTI combination is impossible because cognition =/= the things that motivate us. That said, we can clearly observe many types being more common than others...|||Sx as hell...   https://www.youtube.com/watch?v=uelHwf8o7_U|||Very seriously toying with the idea of being 351. I can be pretty aggressive and controlling, but I have a massive 'holier than thou' streak once you get to know me with any depth (like more than 1-2...|||I'd say it's a mostly emotional experience.|||Because I find in depth study of cognition more interesting. Included MBTI in my answer to illustrate how much I enjoyed socionics. Interpret the post as wholehearted agreement with what she was...|||No real argument here.   I know what I am, and what I am is an asshole that is, for a second baring his teeth at the people that seemingly have a lot to say but refuse to address me beyond a single...|||I would refer you to the OP. If you need a literal analysis, I'm happy to provide it. That said, I'm going to assume you're able to read and interpret symbolism, so I doubt you'll need that much help.|||And this was sort of my thought process. The initial post and how it connects the generally accepted view of the type 3 and what the type 3 is motivated by doesn't seem to be in need of explanation....|||Oh my...  I am quite crushable, huh?  Back atcha.|||I was, but it's 5am. I'll take what I can get.|||I wanted to come up with something clever, but it's almost 5am.  I would give you a lot of sex.|||So then why do you even bother asking if you've already made up your mind that you are an extravert, intuitive and feeling? If you've  already decided you are these things and have evidently done the...|||Wasn't sure I listened to sx music until now.   https://www.youtube.com/watch?v=7hcYx_y5xdo|||INFP messaged me with a picture of herself before going to bed.  Asked me to say goodnight to her.  Asked me to say goodnight to her again.  I did. I thought it was weird, secretly.  I like...|||Either way, you see my point. The actual rarity is skewed and makes a type seem rarer than it likely is in real contexts. I may have been the only one of my type in a class room in High School, but...|||WOW my math was off! I literally just said numbers. See how over glorified the type is?  Even so, though. I saw around two-hundred people (REALLY rough estimate) between both of my jobs. It's...|||Rarity in the context of the entire planet is somewhat silly. Even if we assume the greatest reports of our 'rarity', we (INTJ) encompass 1% of the planet, which sounds rare when you don't think...|||One could say that whatever the culmination of all of our actions are throughout our lives equal our potential.  In this sense, everybody does, but as we all know, not everyone's potential is the...|||I would agree, however, those that profess to know their type not stating anything about their type's motivations strike me as either not wanting to touch on them or not being able to. If they don't...|||Lol.'"
input_arr = []
input_arr.append(input_string)

In [255]:
history = model.fit(partial_feature_train,
                    partial_m_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(feature_val, m_val),
                    verbose=0)

model.save("Mind.h5")

In [256]:
res1 = model.predict(feature_train)
minima = float((min(res1))[0])
maxima = float((max(res1))[0])
range = maxima-minima

In [257]:
results = model.predict(input_arr)
mind_res = (float_to_mind(normalize(results, range)))[0]

In [258]:
history = model.fit(partial_feature_train,
                    partial_e_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(feature_val, e_val),
                    verbose=0)
model.save("Energy.h5")

In [259]:
res1 = model.predict(feature_train)
minima = float((min(res1))[0])
maxima = float((max(res1))[0])
range = maxima-minima

In [260]:
results = model.predict(input_arr)
energy_res = (float_to_energy(normalize(results, range)))[0]

In [261]:
history = model.fit(partial_feature_train,
                    partial_n_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(feature_val, n_val),
                    verbose=0)
model.save("Nature.h5")

In [262]:
res1 = model.predict(feature_train)
minima = float((min(res1))[0])
maxima = float((max(res1))[0])
range = maxima-minima

In [263]:
results = model.predict(input_arr)
nature_res = (float_to_nature(normalize(results, range)))[0]

In [264]:
history = model.fit(partial_feature_train,
                    partial_t_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(feature_val, t_val),
                    verbose=0)
model.save("Tactics.h5")

In [265]:
res1 = model.predict(feature_train)
minima = float((min(res1))[0])
maxima = float((max(res1))[0])
range = maxima-minima

In [266]:
results = model.predict(input_arr)
tactics_res = (float_to_tactics(normalize(results, range)))[0]

In [267]:
print(mind_res + energy_res + nature_res + tactics_res)

ISTJ
